In [ ]:
!pip install arch
!pip install yfinance
!pip install mgarch
!pip install qpsolvers
!pip install get_quote_yahoo
!pip install --upgrade pandas-datareader
!pip install --upgrade pandas
!pip install quadprog
!pip install --upgrade scipy==1.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement get_quote_yahoo (from versions: none)
ERROR: No matching distribution found for get_quote_yahoo
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, http

In [ ]:
#Importing packages
import numpy as np
import pandas as pd
#Using the yahoo finance api to grab data
import yfinance as yf
import qpsolvers as qp #Quadratic Programming package
from pandas_datareader import data #To get marketcap data from yahoo and other data points
from numpy.linalg import multi_dot #To multiply more than 2 matrices
from sklearn.preprocessing import StandardScaler #To scale data
import arch #to use Garch(1,1)
import matplotlib.pyplot as plt
from pandas_datareader import data
import mgarch

In [ ]:
#Need to multiply market caps by 1,000,000
Data = pd.read_excel('ESG Data(Cleaned).xlsx')
# splitting data into 75% training and 25% testing
Training_Data = Data.iloc[:4500,:]
Testing_Data = Data.iloc[4500:,:]
Testing_Data.tail()

,Dates,AAPL Market Cap,AAPL,AAPL ESG Score,F Market Cap,F,F ESG Score,CVS Market Cap,CVS,CVS ESG Score,CVX Market Cap,CVX,CVX ESG Score,SPY
5985,2022-12-13,2.314154e+12,145.47,66.0,5.455810e+10,13.57,45.0,1.329603e+11,101.19,6.0,3.355443e+11,173.53,48.0,401.97
5986,2022-12-14,2.278202e+12,143.21,66.0,5.419626e+10,13.48,45.0,1.322902e+11,100.68,6.0,3.332239e+11,172.33,48.0,399.40
5987,2022-12-15,2.171458e+12,136.50,66.0,5.238704e+10,13.03,45.0,1.282432e+11,97.60,6.0,3.307295e+11,171.04,48.0,389.63
5988,2022-12-16,2.139801e+12,134.51,66.0,4.872839e+10,12.12,45.0,1.254181e+11,95.45,6.0,3.262435e+11,168.72,48.0,383.27
5989,2022-12-19,2.105758e+12,132.37,66.0,4.699958e+10,11.69,45.0,1.248926e+11,95.05,6.0,3.284865e+11,169.88,48.0,380.02


In [ ]:
#Setting the index in the dataset
Data.set_index('Dates')
Training_Data.set_index('Dates')
Testing_Data.set_index('Dates')

#Splitting the S&P500 data between training and test sets
Benchmark = Data['SPY']
Benchmark_Training = Training_Data['SPY']
Benchmark_Testing = Testing_Data['SPY']

#Splitting the stock data between training and test sets excluding S&P500, tbill rates, and Robecosam Scores
Stocks = Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
Stocks_Training = Training_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
Stocks_Testing = Testing_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY','AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
N = len(Stocks.columns)
LastClosingPrice = Stocks.to_numpy()[-1] #Converted to an array to the get the last value
tickers = ['AAPL','F','CVS','CVX']

In [ ]:
Training_Data

,Dates,AAPL Market Cap,AAPL,AAPL ESG Score,F Market Cap,F,F ESG Score,CVS Market Cap,CVS,CVS ESG Score,CVX Market Cap,CVX,CVX ESG Score,SPY
0,2000-01-04,1.648190e+10,0.915,NaN,6.114918e+10,48.6339,NaN,1.469261e+10,18.7500,NaN,5.488024e+10,41.8125,NaN,139.75
1,2000-01-05,1.672310e+10,0.929,NaN,6.137566e+10,48.8140,NaN,1.550071e+10,19.7813,NaN,5.586464e+10,42.5625,NaN,140.00
2,2000-01-06,1.527590e+10,0.848,NaN,6.145115e+10,48.8740,NaN,1.439876e+10,18.3750,NaN,5.824361e+10,44.3750,NaN,137.75
3,2000-01-07,1.599950e+10,0.888,NaN,6.598072e+10,52.4765,NaN,1.474159e+10,18.8125,NaN,5.926902e+10,45.1563,NaN,145.75
4,2000-01-10,1.571810e+10,0.873,NaN,6.477284e+10,51.5159,NaN,1.471710e+10,18.7813,NaN,5.766937e+10,43.9375,NaN,146.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,2017-03-28,7.544525e+11,35.950,70.0,4.630056e+10,11.6500,42.0,8.063024e+10,78.6100,22.0,2.028839e+11,107.1700,40.0,235.32
4496,2017-03-29,7.561314e+11,36.030,70.0,4.641979e+10,11.6800,42.0,8.078410e+10,78.7600,22.0,2.045876e+11,108.0700,40.0,235.54
4497,2017-03-30,7.551345e+11,35.983,70.0,4.641979e+10,11.6800,42.0,8.087641e+10,78.8500,22.0,2.040386e+11,107.7800,40.0,236.29
4498,2017-03-31,7.537179e+11,35.915,70.0,4.634929e+10,11.6400,42.0,8.130960e+10,78.5000,22.0,2.032625e+11,107.3700,40.0,235.74


In [ ]:
m = len(Stocks_Training)
Stocks_Training

,AAPL,F,CVS,CVX
Dates,,,,
2000-01-04,0.915,48.6339,18.7500,41.8125
2000-01-05,0.929,48.8140,19.7813,42.5625
2000-01-06,0.848,48.8740,18.3750,44.3750
2000-01-07,0.888,52.4765,18.8125,45.1563
2000-01-10,0.873,51.5159,18.7813,43.9375
...,...,...,...,...
2017-03-28,35.950,11.6500,78.6100,107.1700
2017-03-29,36.030,11.6800,78.7600,108.0700
2017-03-30,35.983,11.6800,78.8500,107.7800


In [ ]:
#Returns on training data for the stocks
rts_training = (Stocks_Training.to_numpy()[-1] - Stocks_Training.to_numpy()[0])/Stocks_Training.to_numpy()[0]

#Returns on training data for the S&P500
rts_training_benchmark = (Benchmark_Training.to_numpy()[-1] - Benchmark_Training.to_numpy()[0])/Benchmark_Training.to_numpy()[0]

#Choosing BL views by getting daily averages of historical returns as our views.
v1 = (rts_training[0]-rts_training[1])/m #Stock1 will outperform stock2 by v1
v2 = (rts_training[1]-rts_training[2])/m
v3 = (rts_training[2]-rts_training[3])/m
v1

0.008672681825214672

In [ ]:
#Amount it will outperform the other ticker etc.
views = [v1,v2,v3]

#which ones do you view to outperform vs the other...
views2 = np.array([[1, -1,0,0], [0, 1,-1,0],[0, 0,1,-1]])#Each input is w.r.t the alphabetical order of the tickers. See "Tickers" output below to know which one you are assigning your view to

rts = (Stocks_Training.iloc[1:,:].values - Stocks_Training.iloc[:-1,:].values)/Stocks_Training.iloc[:-1,:].values
rts_Benchmark=(Benchmark_Training.iloc[1:].values-Benchmark_Training.iloc[:-1].values)/Benchmark_Training.iloc[:-1].values
#Computing GARCH Covariance
vol = mgarch.mgarch()
vol.fit(rts)
nquarts = 1 #How many quarters in the future do you want to estimate the covariance
cov_nextday = vol.predict(nquarts)
cov = cov_nextday['cov']
ER = rts_Benchmark #Stocks excess returns.
ER_avg = np.mean(ER)#Historical averages of excess returns
hist_avg = np.mean(rts,axis = 0) #Historical averages of log returns for each ticker
varBenchmark = np.var(rts_Benchmark)
A = ER_avg/varBenchmark #(E[r_m]-r_f)/(sigma_m)^2 where r_m is the market returns and r_f is the risk free rate(i.e. the price of risk, risk aversion parameter)

#Getting marketcap data from yahoo finance for the stock tickers only. This gets current marketCap data
marketcaps = data.get_quote_yahoo(Stocks_Training.columns)['marketCap']# Need to get accurate marketCap data on the start date
marketWeights = marketcaps/np.sum(marketcaps)

pi = A*np.matmul(cov,marketWeights).reshape(len(Stocks_Training.columns),1) #Implied Equilibrium Excess Returns
Q = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
P = np.ones((len(Q),len(Stocks_Training.columns)))*views2 #Goes in accordance with the view matrix, Q.
tau = .025 #According to what Black-Litterman suggests tau should be. Should read into why tau should be that value
omega = tau*multi_dot([P,cov,np.transpose(P)])
#first part of the expected rets: [(t*cov)^-1 + P^T*Omega^-1*P]^-1
firstPart = np.linalg.inv(np.linalg.inv(tau*cov) + multi_dot([np.transpose(P),np.linalg.inv(omega),P]))
secondPart = multi_dot([np.linalg.inv(tau*cov),pi]) + multi_dot([np.transpose(P),np.linalg.inv(omega),Q])
# Need to reshape res to be (N,) to then be used as the input with the correct dimensions in the ESG.result() function
res = np.reshape(multi_dot([firstPart,secondPart]),(N,))
df = pd.DataFrame(res,columns = ['BL Expected Returns'],index = Stocks_Training.columns)
df['Historical Averages'] = hist_avg
df

,BL Expected Returns,Historical Averages
AAPL,0.003571,0.001174
F,-0.000809,0.000059
CVS,-0.000377,0.000489
CVX,-0.000550,0.000339


In [ ]:
!pip install qpsolvers
import numpy as np
from qpsolvers import solve_qp

M = np.array([[1.0, 2.0, 0.0], [-8.0, 3.0, 2.0], [0.0, 1.0, 1.0]])
P = M.T @ M  # this is a positive definite matrix
q = np.array([3.0, 2.0, 3.0]) @ M
G = np.array([[1.0, 2.0, 1.0], [2.0, 0.0, 1.0], [-1.0, 2.0, -1.0]])
h = np.array([3.0, 2.0, -2.0])
A = np.array([1.0, 1.0, 1.0])
b = np.array([1.0])

ub =1
lb = 0
x = solve_qp(P, q, G, h, A, b,lb,ub, solver="quadprog")
print(f"QP solution: x = {x}")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


TypeError: ignored

In [ ]:
!pip install qpsolvers
import numpy as np
from qpsolvers import solve_qp

M = np.array([[1.0, 2.0, 0.0], [-8.0, 3.0, 2.0], [0.0, 1.0, 1.0]])
P = cov #The view matrix. nx1 vector where n is the number of views.

A = np.ones((1,N))
c = np.reshape(res,(N,1))
q =  -c.reshape([N,1])
b = np.array([1])
G = np.array([[50,51,0,53]])
h = np.asarray([25], dtype=float)




ub =1
lb = 0


x = solve_qp(P, q, G, h, A, b,lb,ub, solver="cvxopt")
print(f"QP solution: x = {x}")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


TypeError: ignored

In [ ]:
c = np.reshape(res,(N,1))
Q = cov

In [ ]:
G = np.array([[100-ESGSCORE,51,0,53]])
h = np.asarray([50], dtype=float)

NameError: ignored

In [ ]:
Q = cov
A = np.ones((1,N))
c = res
c = c.reshape([N,1])
b = np.array([1])
G = np.array([[50,51,0,53]])
h = np.asarray([25], dtype=float)
a1 = A @ np.linalg.inv(Q)@ np.transpose(A)
a2 = A @ np.linalg.inv(Q)@ np.transpose(G)
a3 = G @ np.linalg.inv(Q) @ np.transpose(G)
lambda1 = (-a3 @ (1- A @ np.linalg.inv(Q) @ c) + a2 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
lambda2 = (np.transpose(a2) @ (1- A @ np.linalg.inv(Q) @ c) - a1 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
x = (np.linalg.inv(Q) @ c) -(lambda1.item() * np.linalg.inv(Q) @ np.transpose(A)) - (lambda2.item() * np.linalg.inv(Q) @ np.transpose(G))

In [ ]:
Training_Data = Data.iloc[0:4434]
Testing_Data = Data.iloc[4434:4435]
#Setting the index in the dataset
Data.set_index('Dates')
Training_Data.set_index('Dates')
Testing_Data.set_index('Dates')

#Splitting the S&P500 data between training and test sets
Benchmark = Data['SPY']
Benchmark_Training = Training_Data['SPY']
Benchmark_Testing = Testing_Data['SPY']

#Splitting the stock data between training and test sets excluding S&P500, tbill rates, and Robecosam Scores
Stocks = Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
Stocks_Training = Training_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
Stocks_Testing = Testing_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY','AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
N = len(Stocks.columns)
LastClosingPrice = Stocks.to_numpy()[-1] #Converted to an array to the get the last value
tickers = ['AAPL','F','CVS','CVX']
m = len(Stocks_Training)
#Returns on training data for the stocks
rts_training = (Stocks_Training.to_numpy()[-1] - Stocks_Training.to_numpy()[0])/Stocks_Training.to_numpy()[0]

#Returns on training data for the S&P500
rts_training_benchmark = (Benchmark_Training.to_numpy()[-1] - Benchmark_Training.to_numpy()[0])/Benchmark_Training.to_numpy()[0]

#Choosing BL views by getting daily averages of historical returns as our views.
v1 = (rts_training[0]-rts_training[1])/m #Stock1 will outperform stock2 by v1
v2 = (rts_training[1]-rts_training[2])/m
v3 = (rts_training[2]-rts_training[3])/m
#Amount it will outperform the other ticker etc.
views = [v1,v2,v3]

#which ones do you view to outperform vs the other...
views2 = np.array([[1, -1,0,0], [0, 1,-1,0],[0, 0,1,-1]])#Each input is w.r.t the alphabetical order of the tickers. See "Tickers" output below to know which one you are assigning your view to

rts = (Stocks_Training.iloc[1:,:].values - Stocks_Training.iloc[:-1,:].values)/Stocks_Training.iloc[:-1,:].values
rts_Benchmark=(Benchmark_Training.iloc[1:].values-Benchmark_Training.iloc[:-1].values)/Benchmark_Training.iloc[:-1].values
#Computing GARCH Covariance
vol = mgarch.mgarch()
vol.fit(rts)
nquarts = 1 #How many quarters in the future do you want to estimate the covariance
cov_nextday = vol.predict(nquarts)
cov = cov_nextday['cov']
ER = rts_Benchmark #Stocks excess returns.
ER_avg = np.mean(ER)#Historical averages of excess returns
hist_avg = np.mean(rts,axis = 0) #Historical averages of log returns for each ticker
varBenchmark = np.var(rts_Benchmark)
A = ER_avg/varBenchmark #(E[r_m]-r_f)/(sigma_m)^2 where r_m is the market returns and r_f is the risk free rate(i.e. the price of risk, risk aversion parameter)

#Getting marketcap data from yahoo finance for the stock tickers only. This gets current marketCap data
marketcaps = data.get_quote_yahoo(Stocks_Training.columns)['marketCap']# Need to get accurate marketCap data on the start date
marketWeights = marketcaps/np.sum(marketcaps)

pi = A*np.matmul(cov,marketWeights).reshape(len(Stocks_Training.columns),1) #Implied Equilibrium Excess Returns
Q = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
P = np.ones((len(Q),len(Stocks_Training.columns)))*views2 #Goes in accordance with the view matrix, Q.
tau = .025 #According to what Black-Litterman suggests tau should be. Should read into why tau should be that value
omega = tau*multi_dot([P,cov,np.transpose(P)])
#first part of the expected rets: [(t*cov)^-1 + P^T*Omega^-1*P]^-1
firstPart = np.linalg.inv(np.linalg.inv(tau*cov) + multi_dot([np.transpose(P),np.linalg.inv(omega),P]))
secondPart = multi_dot([np.linalg.inv(tau*cov),pi]) + multi_dot([np.transpose(P),np.linalg.inv(omega),Q])
# Need to reshape res to be (N,) to then be used as the input with the correct dimensions in the ESG.result() function
res = np.reshape(multi_dot([firstPart,secondPart]),(N,))
df = pd.DataFrame(res,columns = ['BL Expected Returns'],index = Stocks_Training.columns)
df['Historical Averages'] = hist_avg
Q = cov
A = np.ones((1,N))
c = res
c = c.reshape([N,1])
b = np.array([1])
G = np.array([[50,51,0,53]])
h = np.asarray([25], dtype=float)
a1 = A @ np.linalg.inv(Q)@ np.transpose(A)
a2 = A @ np.linalg.inv(Q)@ np.transpose(G)
a3 = G @ np.linalg.inv(Q) @ np.transpose(G)
lambda1 = (-a3 @ (1- A @ np.linalg.inv(Q) @ c) + a2 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
lambda2 = (np.transpose(a2) @ (1- A @ np.linalg.inv(Q) @ c) - a1 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
x = (np.linalg.inv(Q) @ c) -(lambda1.item() * np.linalg.inv(Q) @ np.transpose(A)) - (lambda2.item() * np.linalg.inv(Q) @ np.transpose(G))
x

In [ ]:
np.sum(x)

In [ ]:
Data.iloc[4434]['AAPL']
#((((((x[0] /100) *100000)/(Data.iloc[i-1:i]['AAPL']) * Data.iloc[i:i+1]['AAPL'] +
#((((x[1] /100) *100000)/(Data.iloc[i-1:i]['F']) * Data.iloc[i:i+1]['F'] +
#((((x[2] /100) *100000)/(Data.iloc[i-1:i]['CVS']) * Data.iloc[i:i+1]['CVS'] +
#((((x[3] /100) *100000)/(Data.iloc[i-1:i['CVX']) * Data.iloc[i:i+1]['CVX'] ) -100000)/100000






28.955

In [ ]:
#range(start, stop, step)
for i in range(4434,5990,30):
  Data = pd.read_excel('ESG Data.xlsx')
  Training_Data = Data.iloc[0:i]
  Testing_Data = Data.iloc[i:i+1]


  #Setting the index in the dataset
  Data.set_index('Dates')
  Training_Data.set_index('Dates')
  Testing_Data.set_index('Dates')

#Splitting the S&P500 data between training and test sets
  Benchmark = Data['SPY']
  Benchmark_Training = Training_Data['SPY']
  Benchmark_Testing = Testing_Data['SPY']

#Splitting the stock data between training and test sets excluding S&P500, tbill rates, and Robecosam Scores
  Stocks = Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  Stocks_Training = Training_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  Stocks_Testing = Testing_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY','AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  N = len(Stocks.columns)
  LastClosingPrice = Stocks.to_numpy()[-1] #Converted to an array to the get the last value
  tickers = ['AAPL','F','CVS','CVX']
  m = len(Stocks_Training)
#Returns on training data for the stocks
  rts_training = (Stocks_Training.to_numpy()[-1] - Stocks_Training.to_numpy()[0])/Stocks_Training.to_numpy()[0]

#Returns on training data for the S&P500
  rts_training_benchmark = (Benchmark_Training.to_numpy()[-1] - Benchmark_Training.to_numpy()[0])/Benchmark_Training.to_numpy()[0]

#Choosing BL views by getting daily averages of historical returns as our views.
  v1 = (rts_training[0]-rts_training[1])/m #Stock1 will outperform stock2 by v1
  v2 = (rts_training[1]-rts_training[2])/m
  v3 = (rts_training[2]-rts_training[3])/m
#Amount it will outperform the other ticker etc.
  views = [v1,v2,v3]

#which ones do you view to outperform vs the other...
  views2 = np.array([[1, -1,0,0], [0, 1,-1,0],[0, 0,1,-1]])#Each input is w.r.t the alphabetical order of the tickers. See "Tickers" output below to know which one you are assigning your view to

  rts = (Stocks_Training.iloc[1:,:].values - Stocks_Training.iloc[:-1,:].values)/Stocks_Training.iloc[:-1,:].values
  rts_Benchmark=(Benchmark_Training.iloc[1:].values-Benchmark_Training.iloc[:-1].values)/Benchmark_Training.iloc[:-1].values
#Computing GARCH Covariance
  vol = mgarch.mgarch()
  vol.fit(rts)
  nquarts = 1 #How many quarters in the future do you want to estimate the covariance
  cov_nextday = vol.predict(nquarts)
  cov = cov_nextday['cov']
  ER = rts_Benchmark #Stocks excess returns.
  ER_avg = np.mean(ER)#Historical averages of excess returns
  hist_avg = np.mean(rts,axis = 0) #Historical averages of log returns for each ticker
  varBenchmark = np.var(rts_Benchmark)
  A = ER_avg/varBenchmark #(E[r_m]-r_f)/(sigma_m)^2 where r_m is the market returns and r_f is the risk free rate(i.e. the price of risk, risk aversion parameter)

#Getting marketcap data from yahoo finance for the stock tickers only. This gets current marketCap data
  marketcaps = data.get_quote_yahoo(Stocks_Training.columns)['marketCap']# Need to get accurate marketCap data on the start date
  marketWeights = marketcaps/np.sum(marketcaps)

  pi = A*np.matmul(cov,marketWeights).reshape(len(Stocks_Training.columns),1) #Implied Equilibrium Excess Returns
  Q = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
  P = np.ones((len(Q),len(Stocks_Training.columns)))*views2 #Goes in accordance with the view matrix, Q.
  tau = .025 #According to what Black-Litterman suggests tau should be. Should read into why tau should be that value
  omega = tau*multi_dot([P,cov,np.transpose(P)])
#first part of the expected rets: [(t*cov)^-1 + P^T*Omega^-1*P]^-1
  firstPart = np.linalg.inv(np.linalg.inv(tau*cov) + multi_dot([np.transpose(P),np.linalg.inv(omega),P]))
  secondPart = multi_dot([np.linalg.inv(tau*cov),pi]) + multi_dot([np.transpose(P),np.linalg.inv(omega),Q])
# Need to reshape res to be (N,) to then be used as the input with the correct dimensions in the ESG.result() function
  res = np.reshape(multi_dot([firstPart,secondPart]),(N,))
  df = pd.DataFrame(res,columns = ['BL Expected Returns'],index = Stocks_Training.columns)
  df['Historical Averages'] = hist_avg
  Q = cov
  A = np.ones((1,N))
  c = res
  c = c.reshape([N,1])
  b = np.array([1])
  G = np.array([[50,51,0,53]])
  h = np.asarray([25], dtype=float)
  a1 = A @ np.linalg.inv(Q)@ np.transpose(A)
  a2 = A @ np.linalg.inv(Q)@ np.transpose(G)
  a3 = G @ np.linalg.inv(Q) @ np.transpose(G)
  lambda1 = (-a3 @ (1- A @ np.linalg.inv(Q) @ c) + a2 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
  lambda2 = (np.transpose(a2) @ (1- A @ np.linalg.inv(Q) @ c) - a1 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
  x = (np.linalg.inv(Q) @ c) -(lambda1.item() * np.linalg.inv(Q) @ np.transpose(A)) - (lambda2.item() * np.linalg.inv(Q) @ np.transpose(G))
  x

KeyboardInterrupt: ignored

In [ ]:
#range(start, stop, step)

results = []
for i in range(4434,5990,21):
  Data = pd.read_excel('ESG Data(Cleaned).xlsx')
  Training_Data = Data.iloc[0:i]
  Testing_Data = Data.iloc[i:i+1]


  #Setting the index in the dataset
  Data.set_index('Dates')
  Training_Data.set_index('Dates')
  Testing_Data.set_index('Dates')

#Splitting the S&P500 data between training and test sets
  Benchmark = Data['SPY']
  Benchmark_Training = Training_Data['SPY']
  Benchmark_Testing = Testing_Data['SPY']

#Splitting the stock data between training and test sets excluding S&P500, tbill rates, and Robecosam Scores
  Stocks = Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  Stocks_Training = Training_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  Stocks_Testing = Testing_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY','AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  N = len(Stocks.columns)
  LastClosingPrice = Stocks.to_numpy()[-1] #Converted to an array to the get the last value
  tickers = ['AAPL','F','CVS','CVX']
  m = len(Stocks_Training)
#Returns on training data for the stocks
  rts_training = (Stocks_Training.to_numpy()[-1] - Stocks_Training.to_numpy()[0])/Stocks_Training.to_numpy()[0]

#Returns on training data for the S&P500
  rts_training_benchmark = (Benchmark_Training.to_numpy()[-1] - Benchmark_Training.to_numpy()[0])/Benchmark_Training.to_numpy()[0]

#Choosing BL views by getting daily averages of historical returns as our views.
  v1 = (rts_training[0]-rts_training[1])/m #Stock1 will outperform stock2 by v1
  v2 = (rts_training[1]-rts_training[2])/m
  v3 = (rts_training[2]-rts_training[3])/m
#Amount it will outperform the other ticker etc.
  views = [v1,v2,v3]

#which ones do you view to outperform vs the other...
  views2 = np.array([[1, -1,0,0], [0, 1,-1,0],[0, 0,1,-1]])#Each input is w.r.t the alphabetical order of the tickers. See "Tickers" output below to know which one you are assigning your view to

  rts = (Stocks_Training.iloc[1:,:].values - Stocks_Training.iloc[:-1,:].values)/Stocks_Training.iloc[:-1,:].values
  rts_Benchmark=(Benchmark_Training.iloc[1:].values-Benchmark_Training.iloc[:-1].values)/Benchmark_Training.iloc[:-1].values
#Computing GARCH Covariance
  vol = mgarch.mgarch()
  vol.fit(rts)
  nquarts = 1 #How many quarters in the future do you want to estimate the covariance
  cov_nextday = vol.predict(nquarts)
  cov = cov_nextday['cov']
  ER = rts_Benchmark #Stocks excess returns.
  ER_avg = np.mean(ER)#Historical averages of excess returns
  hist_avg = np.mean(rts,axis = 0) #Historical averages of log returns for each ticker
  varBenchmark = np.var(rts_Benchmark)
  A = ER_avg/varBenchmark #(E[r_m]-r_f)/(sigma_m)^2 where r_m is the market returns and r_f is the risk free rate(i.e. the price of risk, risk aversion parameter)

#Getting marketcap data from yahoo finance for the stock tickers only. This gets current marketCap data
  marketcaps = data.get_quote_yahoo(Stocks_Training.columns)['marketCap']# Need to get accurate marketCap data on the start date
  marketWeights = marketcaps/np.sum(marketcaps)

  pi = A*np.matmul(cov,marketWeights).reshape(len(Stocks_Training.columns),1) #Implied Equilibrium Excess Returns
  Q = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
  P = np.ones((len(Q),len(Stocks_Training.columns)))*views2 #Goes in accordance with the view matrix, Q.
  tau = .025 #According to what Black-Litterman suggests tau should be. Should read into why tau should be that value
  omega = tau*multi_dot([P,cov,np.transpose(P)])
#first part of the expected rets: [(t*cov)^-1 + P^T*Omega^-1*P]^-1
  firstPart = np.linalg.inv(np.linalg.inv(tau*cov) + multi_dot([np.transpose(P),np.linalg.inv(omega),P]))
  secondPart = multi_dot([np.linalg.inv(tau*cov),pi]) + multi_dot([np.transpose(P),np.linalg.inv(omega),Q])
# Need to reshape res to be (N,) to then be used as the input with the correct dimensions in the ESG.result() function
  res = np.reshape(multi_dot([firstPart,secondPart]),(N,))
  df = pd.DataFrame(res,columns = ['BL Expected Returns'],index = Stocks_Training.columns)
  df['Historical Averages'] = hist_avg
  Q = cov
  A = np.ones((1,N))
  c = res
  c = c.reshape([N,1])
  b = np.array([1])
  G = np.array([[50,51,0,53]])
  h = np.asarray([25], dtype=float)
  a1 = A @ np.linalg.inv(Q)@ np.transpose(A)
  a2 = A @ np.linalg.inv(Q)@ np.transpose(G)
  a3 = G @ np.linalg.inv(Q) @ np.transpose(G)
  lambda1 = (-a3 @ (1- A @ np.linalg.inv(Q) @ c) + a2 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
  lambda2 = (np.transpose(a2) @ (1- A @ np.linalg.inv(Q) @ c) - a1 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
  x = (np.linalg.inv(Q) @ c) -(lambda1.item() * np.linalg.inv(Q) @ np.transpose(A)) - (lambda2.item() * np.linalg.inv(Q) @ np.transpose(G))



  #print (x)
  z1 = (Data.iloc[i-1]['AAPL'])
  z2 = (Data.iloc[i]['AAPL'])
  f1 = (Data.iloc[i-1]['F'])
  f2 = Data.iloc[i]['F']
  c1 = (Data.iloc[i-1]['CVS'])
  c2 = (Data.iloc[i]['CVS'])
  cv1 = (Data.iloc[i-1]['CVX'])
  cv2 = Data.iloc[i]['CVX']

  value_of_portfolio = ((((x[0]) *100000)/z1) *z2  + (((x[1]) *100000)/f1) *f2 + (((x[2]) *100000)/c1) *c2 + (((x[3]) *100000)/cv1) *cv2)

  rate_of_return = (value_of_portfolio -100000)/100000

  #print (value_of_portfolio)
  #print (rate_of_return)

  results.append(rate_of_return)
  print(results)



[array([-6.98491931e-15])]
[array([-6.98491931e-15]), array([0.00834143])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943]), array([-0.49899128])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943]), array([-0.49899128]), array([0.])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943]), array([-0.49899128]), array([0.]), array([-0.25067836])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943]), array([-0.49899128]), array([0.]), array([-0.25067836]), array([0.01714076])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943]), array([-0.49899128]), array([0.]), array([-0.25067836]), array([0.01714076]), array([-0.2225077])]
[array([-6.98491931e-15])

In [ ]:
z =

y= 27.30537896*100000/140.42 *138.98	 + -6.00565921*100000/11.36 * 11.56	+ -0.79065695*100000/88.25 * 88.50 + -19.5090628*100000/157.14 *157.17
r =  23.72655425*100000/139.50 *134.87 + -6.77401851*100000/13.72	*13.31 + -0.55908728*100000/101.65 *99.56 +	 -15.39344846*100000/185.34	* 177.93
r
y
q = 41.0920909*100000/140.94*142.65 + -13.51147854*100000/13.10*13.12	+ -1.28779841*100000/102.28	*102.53	+ -25.29281395*100000/172.52*173.54
print(y)
print (r)

print(q)

60828.69360499387
104188.12893633614
132524.6934200134


In [ ]:
#range(start, stop, step)

results = []
for i in range(4434,5990,21):
  Data = pd.read_excel('ESG Data(Cleaned).xlsx')
  Training_Data = Data.iloc[0:i]
  Testing_Data = Data.iloc[i:i+1]


  #Setting the index in the dataset
  Data.set_index('Dates')
  Training_Data.set_index('Dates')
  Testing_Data.set_index('Dates')

#Splitting the S&P500 data between training and test sets
  Benchmark = Data['SPY']
  Benchmark_Training = Training_Data['SPY']
  Benchmark_Testing = Testing_Data['SPY']

#Splitting the stock data between training and test sets excluding S&P500, tbill rates, and Robecosam Scores
  Stocks = Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  Stocks_Training = Training_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  Stocks_Testing = Testing_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY','AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  N = len(Stocks.columns)
  LastClosingPrice = Stocks.to_numpy()[-1] #Converted to an array to the get the last value
  tickers = ['AAPL','F','CVS','CVX']
  m = len(Stocks_Training)
#Returns on training data for the stocks
  rts_training = (Stocks_Training.to_numpy()[-1] - Stocks_Training.to_numpy()[0])/Stocks_Training.to_numpy()[0]

#Returns on training data for the S&P500
  rts_training_benchmark = (Benchmark_Training.to_numpy()[-1] - Benchmark_Training.to_numpy()[0])/Benchmark_Training.to_numpy()[0]

#Choosing BL views by getting daily averages of historical returns as our views.
  v1 = (rts_training[0]-rts_training[1])/m #Stock1 will outperform stock2 by v1
  v2 = (rts_training[1]-rts_training[2])/m
  v3 = (rts_training[2]-rts_training[3])/m
#Amount it will outperform the other ticker etc.
  views = [v1,v2,v3]

#which ones do you view to outperform vs the other...
  views2 = np.array([[1, -1,0,0], [0, 1,-1,0],[0, 0,1,-1]])#Each input is w.r.t the alphabetical order of the tickers. See "Tickers" output below to know which one you are assigning your view to

  rts = (Stocks_Training.iloc[1:,:].values - Stocks_Training.iloc[:-1,:].values)/Stocks_Training.iloc[:-1,:].values
  rts_Benchmark=(Benchmark_Training.iloc[1:].values-Benchmark_Training.iloc[:-1].values)/Benchmark_Training.iloc[:-1].values
#Computing GARCH Covariance
  vol = mgarch.mgarch()
  vol.fit(rts)
  nquarts = 1 #How many quarters in the future do you want to estimate the covariance
  cov_nextday = vol.predict(nquarts)
  cov = cov_nextday['cov']
  ER = rts_Benchmark #Stocks excess returns.
  ER_avg = np.mean(ER)#Historical averages of excess returns
  hist_avg = np.mean(rts,axis = 0) #Historical averages of log returns for each ticker
  varBenchmark = np.var(rts_Benchmark)
  A = ER_avg/varBenchmark #(E[r_m]-r_f)/(sigma_m)^2 where r_m is the market returns and r_f is the risk free rate(i.e. the price of risk, risk aversion parameter)

#Getting marketcap data from yahoo finance for the stock tickers only. This gets current marketCap data
  marketcaps = data.get_quote_yahoo(Stocks_Training.columns)['marketCap']# Need to get accurate marketCap data on the start date
  marketWeights = marketcaps/np.sum(marketcaps)

  pi = A*np.matmul(cov,marketWeights).reshape(len(Stocks_Training.columns),1) #Implied Equilibrium Excess Returns
  Q = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
  P = np.ones((len(Q),len(Stocks_Training.columns)))*views2 #Goes in accordance with the view matrix, Q.
  tau = .025 #According to what Black-Litterman suggests tau should be. Should read into why tau should be that value
  omega = tau*multi_dot([P,cov,np.transpose(P)])
#first part of the expected rets: [(t*cov)^-1 + P^T*Omega^-1*P]^-1
  firstPart = np.linalg.inv(np.linalg.inv(tau*cov) + multi_dot([np.transpose(P),np.linalg.inv(omega),P]))
  secondPart = multi_dot([np.linalg.inv(tau*cov),pi]) + multi_dot([np.transpose(P),np.linalg.inv(omega),Q])
# Need to reshape res to be (N,) to then be used as the input with the correct dimensions in the ESG.result() function
  res = np.reshape(multi_dot([firstPart,secondPart]),(N,))
  df = pd.DataFrame(res,columns = ['BL Expected Returns'],index = Stocks_Training.columns)
  df['Historical Averages'] = hist_avg
  Q = cov
  A = np.ones((1,N))
  c = res
  c = c.reshape([N,1])
  b = np.array([1])
  G = np.array([[50,51,0,53]])
  h = np.asarray([25], dtype=float)
  a1 = A @ np.linalg.inv(Q)@ np.transpose(A)
  a2 = A @ np.linalg.inv(Q)@ np.transpose(G)
  a3 = G @ np.linalg.inv(Q) @ np.transpose(G)
  lambda1 = (-a3 @ (1- A @ np.linalg.inv(Q) @ c) + a2 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
  lambda2 = (np.transpose(a2) @ (1- A @ np.linalg.inv(Q) @ c) - a1 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
  x = (np.linalg.inv(Q) @ c) -(lambda1.item() * np.linalg.inv(Q) @ np.transpose(A)) - (lambda2.item() * np.linalg.inv(Q) @ np.transpose(G))



  #print (x)
  z1 = (Data.iloc[i-1]['AAPL'])
  z2 = (Data.iloc[i]['AAPL'])
  f1 = (Data.iloc[i-1]['F'])
  f2 = Data.iloc[i]['F']
  c1 = (Data.iloc[i-1]['CVS'])
  c2 = (Data.iloc[i]['CVS'])
  cv1 = (Data.iloc[i-1]['CVX'])
  cv2 = Data.iloc[i]['CVX']

  value_of_portfolio = ((((x[0]) *100000)/z1) *z2  + (((x[1]) *100000)/f1) *f2 + (((x[2]) *100000)/c1) *c2 + (((x[3]) *100000)/cv1) *cv2)

  rate_of_return = (value_of_portfolio -100000)/100000

  #print (value_of_portfolio)
  #print (rate_of_return)

  results.append(rate_of_return)
  print(results)



[array([-6.98491931e-15])]
[array([-6.98491931e-15]), array([0.00834143])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943]), array([-0.49899128])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943]), array([-0.49899128]), array([0.])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943]), array([-0.49899128]), array([0.]), array([-0.25067836])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943]), array([-0.49899128]), array([0.]), array([-0.25067836]), array([0.01714076])]
[array([-6.98491931e-15]), array([0.00834143]), array([0.18193222]), array([0.02412943]), array([-0.49899128]), array([0.]), array([-0.25067836]), array([0.01714076]), array([-0.2225077])]
[array([-6.98491931e-15])

In [ ]:
Data.iloc[4434:5990]




,Dates,AAPL Market Cap,AAPL,AAPL ESG Score,F Market Cap,F,F ESG Score,CVS Market Cap,CVS,CVS ESG Score,CVX Market Cap,CVX,CVX ESG Score,SPY
4434,2017-01-02,6.175885e+11,28.955,70.0,4.820116e+10,12.13,42.0,8.415272e+10,78.91,22.0,2.221904e+11,117.70,40.0,223.53
4435,2017-01-03,6.193482e+11,29.038,70.0,5.002907e+10,12.59,42.0,8.568839e+10,80.35,22.0,2.224736e+11,117.85,40.0,225.24
4436,2017-01-04,6.186550e+11,29.005,70.0,5.233382e+10,13.17,42.0,8.504853e+10,79.75,22.0,2.224170e+11,117.82,40.0,226.58
4437,2017-01-05,6.218010e+11,29.153,70.0,5.074434e+10,12.77,42.0,8.682948e+10,81.42,22.0,2.214542e+11,117.31,40.0,226.40
4438,2017-01-06,6.199491e+11,29.478,70.0,5.070460e+10,12.76,42.0,8.766130e+10,82.20,22.0,2.205670e+11,116.84,40.0,227.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5985,2022-12-13,2.314154e+12,145.470,66.0,5.455810e+10,13.57,45.0,1.329603e+11,101.19,6.0,3.355443e+11,173.53,48.0,401.97
5986,2022-12-14,2.278202e+12,143.210,66.0,5.419626e+10,13.48,45.0,1.322902e+11,100.68,6.0,3.332239e+11,172.33,48.0,399.40
5987,2022-12-15,2.171458e+12,136.500,66.0,5.238704e+10,13.03,45.0,1.282432e+11,97.60,6.0,3.307295e+11,171.04,48.0,389.63
5988,2022-12-16,2.139801e+12,134.510,66.0,4.872839e+10,12.12,45.0,1.254181e+11,95.45,6.0,3.262435e+11,168.72,48.0,383.27


In [ ]:
#range(start, stop, step)

results = []
for i in range(4434,5990,21):
  Data = pd.read_excel('ESG Data(Cleaned).xlsx')
  Training_Data = Data.iloc[0:i]
  Testing_Data = Data.iloc[i:i+1]


  #Setting the index in the dataset
  Data.set_index('Dates')
  Training_Data.set_index('Dates')
  Testing_Data.set_index('Dates')

#Splitting the S&P500 data between training and test sets
  Benchmark = Data['SPY']
  Benchmark_Training = Training_Data['SPY']
  Benchmark_Testing = Testing_Data['SPY']

#Splitting the stock data between training and test sets excluding S&P500, tbill rates, and Robecosam Scores
  Stocks = Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  Stocks_Training = Training_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY', 'AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  Stocks_Testing = Testing_Data.drop(['AAPL ESG Score','F ESG Score',
                                        'CVS ESG Score','CVX ESG Score','SPY','AAPL Market Cap','F Market Cap','CVS Market Cap','CVX Market Cap'], axis = 1).set_index('Dates')
  N = len(Stocks.columns)
  LastClosingPrice = Stocks.to_numpy()[-1] #Converted to an array to the get the last value
  tickers = ['AAPL','F','CVS','CVX']
  m = len(Stocks_Training)
#Returns on training data for the stocks
  rts_training = (Stocks_Training.to_numpy()[-1] - Stocks_Training.to_numpy()[0])/Stocks_Training.to_numpy()[0]

#Returns on training data for the S&P500
  rts_training_benchmark = (Benchmark_Training.to_numpy()[-1] - Benchmark_Training.to_numpy()[0])/Benchmark_Training.to_numpy()[0]

#Choosing BL views by getting daily averages of historical returns as our views.
  v1 = (rts_training[0]-rts_training[1])/m #Stock1 will outperform stock2 by v1
  v2 = (rts_training[1]-rts_training[2])/m
  v3 = (rts_training[2]-rts_training[3])/m
#Amount it will outperform the other ticker etc.
  views = [v1,v2,v3]

#which ones do you view to outperform vs the other...
  views2 = np.array([[1, -1,0,0], [0, 1,-1,0],[0, 0,1,-1]])#Each input is w.r.t the alphabetical order of the tickers. See "Tickers" output below to know which one you are assigning your view to

  rts = (Stocks_Training.iloc[1:,:].values - Stocks_Training.iloc[:-1,:].values)/Stocks_Training.iloc[:-1,:].values
  rts_Benchmark=(Benchmark_Training.iloc[1:].values-Benchmark_Training.iloc[:-1].values)/Benchmark_Training.iloc[:-1].values
#Computing GARCH Covariance
  vol = mgarch.mgarch()
  vol.fit(rts)
  nquarts = 1 #How many quarters in the future do you want to estimate the covariance
  cov_nextday = vol.predict(nquarts)
  cov = cov_nextday['cov']
  ER = rts_Benchmark #Stocks excess returns.
  ER_avg = np.mean(ER)#Historical averages of excess returns
  hist_avg = np.mean(rts,axis = 0) #Historical averages of log returns for each ticker
  varBenchmark = np.var(rts_Benchmark)
  A = ER_avg/varBenchmark #(E[r_m]-r_f)/(sigma_m)^2 where r_m is the market returns and r_f is the risk free rate(i.e. the price of risk, risk aversion parameter)

#Getting marketcap data from yahoo finance for the stock tickers only. This gets current marketCap data
  marketcaps = data.get_quote_yahoo(Stocks_Training.columns)['marketCap']# Need to get accurate marketCap data on the start date
  marketWeights = marketcaps/np.sum(marketcaps)

  pi = A*np.matmul(cov,marketWeights).reshape(len(Stocks_Training.columns),1) #Implied Equilibrium Excess Returns
  Q = np.array(views).reshape((len(views),1)) #The view matrix. nx1 vector where n is the number of views.
  P = np.ones((len(Q),len(Stocks_Training.columns)))*views2 #Goes in accordance with the view matrix, Q.
  tau = .025 #According to what Black-Litterman suggests tau should be. Should read into why tau should be that value
  omega = tau*multi_dot([P,cov,np.transpose(P)])
#first part of the expected rets: [(t*cov)^-1 + P^T*Omega^-1*P]^-1
  firstPart = np.linalg.inv(np.linalg.inv(tau*cov) + multi_dot([np.transpose(P),np.linalg.inv(omega),P]))
  secondPart = multi_dot([np.linalg.inv(tau*cov),pi]) + multi_dot([np.transpose(P),np.linalg.inv(omega),Q])
# Need to reshape res to be (N,) to then be used as the input with the correct dimensions in the ESG.result() function
  res = np.reshape(multi_dot([firstPart,secondPart]),(N,))
  df = pd.DataFrame(res,columns = ['BL Expected Returns'],index = Stocks_Training.columns)
  df['Historical Averages'] = hist_avg
  Q = cov
  A = np.ones((1,N))
  c = res
  c = c.reshape([N,1])
  b = np.array([1])
  G = np.array([[50,51,0,53]])
  h = np.asarray([25], dtype=float)
  a1 = A @ np.linalg.inv(Q)@ np.transpose(A)
  a2 = A @ np.linalg.inv(Q)@ np.transpose(G)
  a3 = G @ np.linalg.inv(Q) @ np.transpose(G)
  lambda1 = (-a3 @ (1- A @ np.linalg.inv(Q) @ c) + a2 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
  lambda2 = (np.transpose(a2) @ (1- A @ np.linalg.inv(Q) @ c) - a1 @ (h - G @ np.linalg.inv(Q) @ c ))/(a1@a3- a2@a2)
  x = (np.linalg.inv(Q) @ c) -(lambda1.item() * np.linalg.inv(Q) @ np.transpose(A)) - (lambda2.item() * np.linalg.inv(Q) @ np.transpose(G))



  #print (x)
  z1 = (Data.iloc[i-1]['AAPL'])
  z2 = (Data.iloc[i]['AAPL'])
  f1 = (Data.iloc[i-1]['F'])
  f2 = Data.iloc[i]['F']
  c1 = (Data.iloc[i-1]['CVS'])
  c2 = (Data.iloc[i]['CVS'])
  cv1 = (Data.iloc[i-1]['CVX'])
  cv2 = Data.iloc[i]['CVX']

  value_of_portfolio = ((((x[0]) *100000)/z1) *z2  + (((x[1]) *100000)/f1) *f2 + (((x[2]) *100000)/c1) *c2 + (((x[3]) *100000)/cv1) *cv2)

  rate_of_return = (value_of_portfolio -100000)/100000

  #print (value_of_portfolio)
  #print (rate_of_return)

  results.append(rate_of_return)
  print(results)



In [ ]:
q =


#[[ 35.95069136]
 #[-10.85689122]
 #[ -1.0969489 ]
 #[-22.99685124]]

(35.95069136 *100000)/Data.iloc[5978]['AAPL'] #* Data.iloc[5979]['AAPL'] + (-10.85689122 *100000/Data.iloc[5978]['F'] * Data.iloc[5979]['F']) + -1.0969489*100000/Data.iloc[5978]['CVS'] *Data.iloc[5979]['CVS']
#+ -22.99685124 *100000/Data.iloc[5978]['CVX'] *Data.iloc[5979]['CVX']


24322.232162911845

In [ ]:
 results = []
for i in range(10):

  results.append(i)
  print (results)

[0]
[0, 1]
[0, 1, 2]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 5]
[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 5, 6, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
